In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output, display
from dotenv import load_dotenv
from Website.models import UserDefaultSettings, UserDynamicPreferences, db
import os




In [2]:
app = Flask(__name__)

load_dotenv()

# Configuring the database URI from the .env file
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Initialize SQLAlchemy with the app
db.init_app(app)

In [3]:
def get_fan_counts():
    # Perform the query to get fan counts grouped by current_favorite
    query = db.session.query(
        UserDynamicPreferences.current_favorite,
        db.func.count(UserDynamicPreferences.current_favorite).label('fan_count')
    ).group_by(UserDynamicPreferences.current_favorite)
    
    # Convert query result to Pandas DataFrame
    result = query.all()
    df = pd.DataFrame(result, columns=['current_favorite', 'fan_count'])
    return df

In [4]:
def reset_session():
    db.session.remove()
    db.session.configure(bind=db.engine)  # Rebind the engine if necessary

def get_fan_counts():
    reset_session()  # Reset the session to avoid stale data
    
    # Perform the query to get fan counts grouped by current_favorite
    query = db.session.query(
        UserDynamicPreferences.current_favorite,
        db.func.count(UserDynamicPreferences.current_favorite).label('fan_count')
    ).group_by(UserDynamicPreferences.current_favorite)
    
    result = query.all()

    # Convert query result to Pandas DataFrame
    df = pd.DataFrame(result, columns=['current_favorite', 'fan_count'])

    # Debugging: print the result of the query
    print("Query Result:")
    print(df)

    return df


In [5]:
with app.app_context():
    df = get_fan_counts()
    print(df)  # Check if data is being fetched



Query Result:
    current_favorite  fan_count
0         Tony Stark       2496
1  Sabrina Carpenter       2426
2         Snoop Dogg       2573
3       LeBron James       2505
    current_favorite  fan_count
0         Tony Stark       2496
1  Sabrina Carpenter       2426
2         Snoop Dogg       2573
3       LeBron James       2505


: 

# Run Event Sim

In [6]:
import matplotlib
matplotlib.use('TkAgg')  # Force a GUI backend (if not already set)

def display_real_time_plot(refresh_interval=5):
    plt.ion()  # Turn on interactive mode
    fig, ax = plt.subplots()

    # Initialize the plot
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Number of Fans')
    ax.set_title('Real-Time Fan Count of Celebrities')
    ax.set_ylim(0, 6000)  # Adjust according to expected fan counts
    ax.set_xlim(0, 1000)

    # To track data over time
    time_data = []
    fan_counts_data = {celebrity: [] for celebrity in ['Sabrina Carpenter', 'Snoop Dogg', 'Tony Stark', 'LeBron James']}

    # Initialize empty plot lines for each celebrity
    lines = {}
    for celebrity in fan_counts_data.keys():
        lines[celebrity], = ax.plot([], [], label=celebrity)

    ax.legend(loc='upper left')

    try:
        start_time = time.time()
        while True:
            current_time = time.time() - start_time
            time_data.append(current_time)

            # Fetch the latest fan counts
            df = get_fan_counts()
            print(df)  # Debug: print to check if data is being returned

            # Update the plot data for each celebrity
            for celebrity in fan_counts_data.keys():
                fan_count = df[df['current_favorite'] == celebrity]['fan_count'].sum() if not df[df['current_favorite'] == celebrity].empty else 0
                fan_counts_data[celebrity].append(fan_count)

            # Update plot data
            for celebrity, line in lines.items():
                line.set_data(time_data, fan_counts_data[celebrity])

            # Rescale the x-axis and y-axis dynamically
            ax.relim()
            ax.autoscale_view()

            plt.draw()
            plt.pause(0.1)  # Pause to allow real-time updates

            time.sleep(refresh_interval)

    except KeyboardInterrupt:
        print("Real-time plot update interrupted.")
    finally:
        plt.ioff()
        plt.show()  # Keep the final plot visible after interrupt

# Main script to display real-time plot
if __name__ == '__main__':
    with app.app_context():
        display_real_time_plot(refresh_interval=5)  # Update every 5 seconds


Query Result:
    current_favorite  fan_count
0         Tony Stark       2496
1  Sabrina Carpenter       2426
2         Snoop Dogg       2573
3       LeBron James       2505
    current_favorite  fan_count
0         Tony Stark       2496
1  Sabrina Carpenter       2426
2         Snoop Dogg       2573
3       LeBron James       2505
Query Result:
    current_favorite  fan_count
0         Tony Stark       2496
1  Sabrina Carpenter       2426
2         Snoop Dogg       2573
3       LeBron James       2505
    current_favorite  fan_count
0         Tony Stark       2496
1  Sabrina Carpenter       2426
2         Snoop Dogg       2573
3       LeBron James       2505
